<a href="https://colab.research.google.com/github/ericbonelli/Cientista-de-Dados_EBAC/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌳 Random Forest

## ✅ 1. O que é Random Forest?

O **Random Forest** é um algoritmo de aprendizado de máquina que constrói várias árvores de decisão diferentes e combina os resultados delas para fazer uma previsão final.

A diferença principal em relação ao Bagging é que o Random Forest **além de fazer amostragem com reposição (bootstrap)**, também **escolhe aleatoriamente apenas algumas features (colunas)** para cada árvore. Isso diminui a correlação entre elas e aumenta a robustez do modelo.

Como resultado, temos um modelo mais **preciso, estável e resistente ao overfitting** do que uma única árvore de decisão.

---



## ✅ 2. Passo a passo para o algoritmo Random Forest


1. **Bootstrap**
   - Criar várias amostras com reposição a partir do conjunto de dados original.

2. **Seleção aleatória de features**
   - Para cada árvore, escolher aleatoriamente um subconjunto de colunas (features) para treinar, o que gera árvores diferentes e menos correlacionadas.

3. **Modelagem com Decision Trees**
   - Treinar uma árvore de decisão em cada amostra com as features selecionadas.

4. **Agregação**
   - Combinar as previsões das árvores:
     - Para classificação: usar votação (classe mais votada).
     - Para regressão: usar a média das previsões.

   ---

## ✅ 3. Qual a diferença entre Bagging e Random Forest?

| Característica              | **Bagging**                              | **Random Forest**                                 |
|----------------------------|------------------------------------------|---------------------------------------------------|
| Tipo de modelo base        | Qualquer (mas geralmente Decision Trees) | Apenas Árvores de Decisão                        |
| Bootstrap (amostragem)     | ✅ Sim                                   | ✅ Sim                                            |
| Seleção aleatória de features | ❌ Não (usa todas)                      | ✅ Sim (subset aleatório por árvore)             |
| Correlação entre árvores   | Alta                                     | Baixa (por causa da seleção de features)         |
| Votação/Média              | ✅ Sim                                   | ✅ Sim                                            |
| Exemplo em sklearn         | `BaggingClassifier`                      | `RandomForestClassifier`                         |

---

## ✅ 4. Implementação em Python

Vamos usar o conjunto de dados Iris como exemplo e aplicar o Randon Farest, sendo demonstrando passo a passo de forma manual e depois usando scikit-learn

---

# 🌳 4.1. Random Forest com Dataset Iris - Manual

Vamos entender o funcionamento do Randon forest recriando suas 4 etapas principais com o dataset Iris.

---

## 📌 Etapa 1: Bootstrap (amostragem com reposição)

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# Carregar dataset Iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target)
feature_names = X.columns.tolist()

# Dividir dados para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## 🔍 Etapa 2: Feature Selection (seleção aleatória de colunas)

In [5]:
# Função para selecionar aleatoriamente k features
def selecionar_features(X, k, random_state=None):
    np.random.seed(random_state)
    cols = np.random.choice(X.columns, size=k, replace=False)
    return X[cols], cols

## 🌱 Etapa 3: Treinar várias Árvores com subsets diferentes

In [6]:
from sklearn.tree import DecisionTreeClassifier

n_arvores = 10
k_features = 2
modelos = []
colunas_usadas = []

for i in range(n_arvores):
    # Etapa 1: Bootstrap
    X_sample, y_sample = resample(X_train, y_train, replace=True, random_state=i)

    # Etapa 2: Feature Selection
    X_sub, cols = selecionar_features(X_sample, k=k_features, random_state=i)

    # Etapa 3: Modelagem
    modelo = DecisionTreeClassifier(random_state=i)
    modelo.fit(X_sub, y_sample)

    modelos.append(modelo)
    colunas_usadas.append(cols)

## 🧠 Etapa 4: Agregação das Previsões (Votação)


In [7]:
from scipy.stats import mode
from sklearn.metrics import accuracy_score

# Fazer previsões com as árvores
previsoes = []

for modelo, cols in zip(modelos, colunas_usadas):
    X_test_sub = X_test[cols]
    pred = modelo.predict(X_test_sub)
    previsoes.append(pred)

# Votação majoritária
previsoes = np.array(previsoes)
final = mode(previsoes, axis=0, keepdims=True).mode[0]

print("✅ Acurácia do Random Forest manual:", accuracy_score(y_test, final))

✅ Acurácia do Random Forest manual: 1.0


# ⚡ 4.2. Random Forest com Scikit-learn (automático)

---



In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Carregar dados
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Criar Random Forest com 10 árvores
rf = RandomForestClassifier(n_estimators=10, random_state=42)
rf.fit(X_train, y_train)

# Previsão e avaliação
y_pred = rf.predict(X_test)
print("✅ Acurácia com RandomForestClassifier:", accuracy_score(y_test, y_pred))

✅ Acurácia com RandomForestClassifier: 1.0
